# Finding the (math) cultural similarity

When someone thinks about Why humans behave as we do, one of the most common answers is "because of culture". And very often that is the end of the issue. But if one wants to go deeper, you better ask to an ethnologist or read an Ethnology book, because Ethnology is the Social Science that has be trying to go deeper on cultural  problems.

Ethnologic work tends to be a really hard proccess to sistematize large number of data across multiples research sources. Very heterogenouous sources actually, about complex aspect of human activity that we tend to call 'culture'.

Culture is...a polisemic word, which means it has many meanings, as someone has put (Kuper, 1999), culture is battle of meaning. But culture definition is another story. The most important thing that we need to know is that culture is in the realm of meaning and behaviour. So, Ethnology have being using mainly qualitative methodologies to explore it.

Qualitative is often viewed as non-numerical, more related to narratives and humanities. Despite qualitative doesn't involve numbers, it doesn't mean doesn't support mathematical approach. Actually, many Qualitative technics as classification and codification are based on logic and set theory. Many are task are to build networks between people, concepts or genalogic studies, which are all realted to graph theory.

As an Anthopologist I am very familiar and I handle many of these techniques with expertice, but I am also a Data Scientist and I can handle many quantitative and computational techniques. Is from this point of view that I want to apply some "digital humanities" approach to a very old problem in Ethnologic field. I like to thing that we could call this approach a mixed method between quantitative and cualitative.

Today I am going to experiment with this. I will use one of the most classic ethnologic datasets (Murdock Ethnographic Atlas) to compare similarity among cultures. Something that in other times would require a very expert judgement about many aspects of hundreds of cultural groups.

To do so, I will transform each cultural group into a multidimensional vector (Cultural Algebra!) then I will be able to calculate Cosine distance (yes, I will use some geometry and numeric degrees!, god bless us all). Shorter the distance between this abstract vectors, higher the cultural similarity.

The Ethnographic Atlas dataset (ea) contains information about cultural 1241 groups and 94 features, including economic, political, religious, kindship aspect of its social life, all aspect that are part of a board definition of culture.

One could always find many arguments about Why this approach is incomplete, limited or simplistic, but is on hand and no one has done it before. Worth trying.

Enough talking, lets work.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [184]:
ea=pd.read_csv("data.csv")
soc=pd.read_csv("societies.csv")
var=pd.read_csv("variables.csv")
#societies_mapping=pd.read_csv("societies_mapping.csv")
codes=pd.read_csv("codes.csv")

### Explore dataset and its realtions

In [20]:
ea.head()

,soc_id,sub_case,year,var_id,code,comment,references,source_coded_data,admin_comment
0,Aa1,Nyai Nyae region,1950.0,EA001,8.0,NaN,biesele1972; biesele1975; biesele1976; draper1...,EthnographicAtlas_1967_p62,NaN
1,Aa2,with special reference to Central Dorobo,1920.0,EA001,4.0,NaN,huntingford1929; huntingford1942; huntingford1...,EthnographicAtlas_1967_p62,NaN
2,Aa3,Gei/Khauan tribe,1840.0,EA001,1.0,NaN,hoernl1918; hoernl1925; kohler1970; murdock193...,EthnographicAtlas_1967_p62,NaN
3,Aa4,NaN,1920.0,EA001,4.0,NaN,lebzelter1934; schmidt1930; vedder1923,Ethnology_Vol1_No3_Jul_1962,NaN
4,Aa5,"Epulu net-hunters, Ituri forest",1930.0,EA001,3.0,NaN,deleeuwe1966; hart1978; putnam1948; turnbull19...,EthnographicAtlas_1967_p62,NaN


In [11]:
var.head()

,id,category,title,definition,type,units,source,changes,notes
0,EA001,"Subsistence, Economy",Subsistence economy: gathering,Dependence on the gathering of wild plants and...,Ordinal,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN
1,EA002,"Subsistence, Economy",Subsistence economy: hunting,"Dependence on hunting, including trapping and ...",Ordinal,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN
2,EA003,"Subsistence, Economy",Subsistence economy: fishing,"Dependence on fishing, including shellfishing ...",Ordinal,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN
3,EA004,"Subsistence, Economy",Subsistence economy: animal husbandry,"Dependence on animal husbandry, relative to ot...",Ordinal,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN
4,EA005,"Subsistence, Economy",Subsistence economy: agriculture,"Dependence on agriculture, relative to other s...",Ordinal,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN


In [17]:
soc.head()

,id,xd_id,pref_name_for_society,glottocode,ORIG_name_and_ID_in_this_dataset,alt_names_by_society,main_focal_year,HRAF_name_ID,HRAF_link,origLat,origLong,Lat,Long,Comment,glottocode_comment
0,Aa1,xd1,!Kung,juho1239,Kung (Aa1),"Kung Bushmen, !Kung (Was Nyae), Was Nyae",1950.0,San (FX10),http://ehrafworldcultures.yale.edu/collection?...,-20.0,21.0,-20.0,21.0,Original,NaN
1,Ab1,xd10,Herero,here1253,Herero (Ab1),"Damara, Ovaherero",1900.0,NaN,NaN,-21.0,16.0,-21.0,16.0,Original,NaN
2,Ad31,xd100,Duruma,duru1249,Duruma (Ad31),Waduruma,1910.0,NaN,NaN,-4.0,39.0,-4.0,39.0,Original,NaN
3,Ad32,xd101,Giriama,giry1241,Giriama (Ad32),"Kiriama, Wagiryama",1900.0,NaN,NaN,-3.0,40.0,-3.0,40.0,Original,NaN
4,Ad33,xd102,Pokomo,poko1261,Pokomo (Ad33),Wapokomo,1900.0,NaN,NaN,-1.0,40.0,-1.0,40.0,Original,NaN


In [67]:
set([item for lista in var.category.str.split(", ").tolist() for item in lista])

{'Class',
 'Community organization',
 'Ecology',
 'Economy',
 'Games',
 'Gender',
 'Housing',
 'Kinship',
 'Labour',
 'Leadership',
 'Marriage',
 'Politics',
 'Population',
 'Property',
 'Ritual',
 'Settlement',
 'Subsistence',
 'Wealth Transactions'}

In [20]:
from collections import Counter
var.category.str.split(", ").map(Counter).sum()#.value_counts()

Counter({'Class': 6,
         'Community organization': 16,
         'Ecology': 6,
         'Economy': 44,
         'Games': 1,
         'Gender': 15,
         'Housing': 11,
         'Kinship': 25,
         'Labour': 24,
         'Leadership': 4,
         'Marriage': 19,
         'Politics': 12,
         'Population': 2,
         'Property': 4,
         'Ritual': 7,
         'Settlement': 2,
         'Subsistence': 11,
         'Wealth Transactions': 6})

In [4]:
var.type.value_counts()

Categorical    84
Ordinal         9
Continuous      1
Name: type, dtype: int64

In [77]:
var[var.type == 'Continuous'].head()

,id,category,title,definition,type,units,source,changes,notes
93,EA202,Population,Population size,"Population of ethnic group as a whole, unless ...",Continuous,NaN,Murdock (1962-1971); Ember (1992),NaN,NaN


In [31]:
ea[ea.var_id == "EA202"].head()

,soc_id,sub_case,year,var_id,code,comment,references,source_coded_data,admin_comment
20656,Aa1,Nyai Nyae region,1953.0,EA202,3500.0,"Of this total, 1,000 were located in the Nyae ...",NaN,Carol Ember's Concordance Database,NaN
20657,Aa2,with special reference to Central Dorobo,1925.0,EA202,1100.0,221 Central Dorobo in 1942,NaN,EA Bibliography,NaN
20658,Aa3,Gei/Khauan tribe,1876.0,EA202,2500.0,in 1876; See HRAF reference 4:7; note focal ye...,NaN,Carol Ember's Concordance Database,NaN
20659,Aa4,NaN,1946.0,EA202,29000.0,in 1946; note focal year for cultural data is ...,NaN,EA Bibliography,NaN
20660,Aa5,"Epulu net-hunters, Ituri forest",1951.0,EA202,40000.0,in 1951; See HRAF reference 2:26; note focal y...,NaN,Carol Ember's Concordance Database,NaN


In [81]:
var[var.type == 'Categorical'].head(3)

,id,category,title,definition,type,units,source,changes,notes
5,EA006,"Marriage, Kinship, Wealth Transactions, Economy",Transactions at marriage: prevailing type,Prevailing type of transfer or exchange at mar...,Categorical,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN
6,EA007,"Marriage, Kinship, Wealth Transactions, Economy",Transactions at marriage: alternate type,Alternative types of transfer or exchange at m...,Categorical,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN
7,EA008,"Marriage, Kinship",Domestic organization,The prevailing form of domestic or familial or...,Categorical,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN


In [83]:
ea[ea.var_id == "EA007"].head(3)

,soc_id,sub_case,year,var_id,code,comment,references,source_coded_data,admin_comment
90370,Aa1,Nyai Nyae region,1950.0,EA007,8.0,NaN,biesele1972; biesele1975; biesele1976; draper1...,EthnographicAtlas_1967_p62,NaN
90371,Aa2,with special reference to Central Dorobo,1920.0,EA007,8.0,NaN,huntingford1929; huntingford1942; huntingford1...,EthnographicAtlas_1967_p62,NaN
90372,Aa3,Gei/Khauan tribe,1840.0,EA007,1.0,NaN,hoernl1918; hoernl1925; kohler1970; murdock193...,EthnographicAtlas_1967_p62,NaN


In [84]:
ea[ea.var_id == "EA007"].code.value_counts()

8.0    1036
2.0     123
3.0      31
1.0      29
7.0      28
5.0      19
6.0       6
Name: code, dtype: int64

In [80]:
var[var.type == 'Ordinal'].head(3)

,id,category,title,definition,type,units,source,changes,notes
0,EA001,"Subsistence, Economy",Subsistence economy: gathering,Dependence on the gathering of wild plants and...,Ordinal,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN
1,EA002,"Subsistence, Economy",Subsistence economy: hunting,"Dependence on hunting, including trapping and ...",Ordinal,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN
2,EA003,"Subsistence, Economy",Subsistence economy: fishing,"Dependence on fishing, including shellfishing ...",Ordinal,NaN,Murdock (1962-1971); Gray (1999),NaN,NaN


In [36]:
ea[ea.var_id == "EA001"].head()

,soc_id,sub_case,year,var_id,code,comment,references,source_coded_data,admin_comment
0,Aa1,Nyai Nyae region,1950.0,EA001,8.0,NaN,biesele1972; biesele1975; biesele1976; draper1...,EthnographicAtlas_1967_p62,NaN
1,Aa2,with special reference to Central Dorobo,1920.0,EA001,4.0,NaN,huntingford1929; huntingford1942; huntingford1...,EthnographicAtlas_1967_p62,NaN
2,Aa3,Gei/Khauan tribe,1840.0,EA001,1.0,NaN,hoernl1918; hoernl1925; kohler1970; murdock193...,EthnographicAtlas_1967_p62,NaN
3,Aa4,NaN,1920.0,EA001,4.0,NaN,lebzelter1934; schmidt1930; vedder1923,Ethnology_Vol1_No3_Jul_1962,NaN
4,Aa5,"Epulu net-hunters, Ituri forest",1930.0,EA001,3.0,NaN,deleeuwe1966; hart1978; putnam1948; turnbull19...,EthnographicAtlas_1967_p62,NaN


In [16]:
ea[ea.var_id == "EA001"].code.value_counts()

0.0    722
1.0    265
2.0    118
3.0     62
4.0     48
5.0     35
6.0     31
7.0      6
8.0      3
Name: code, dtype: int64

In [86]:
soc2 = soc[['id','pref_name_for_society','glottocode','Lat','Long']]
ea2 = ea[['soc_id','var_id','code']]
var2 = var[['id','title','category']]

In [97]:
soc2.head(2)

,id,pref_name_for_society,glottocode,Lat,Long
0,Aa1,!Kung,juho1239,-20.0,21.0
1,Ab1,Herero,here1253,-21.0,16.0


In [96]:
ea2.head(2)

,soc_id,var_id,code
0,Aa1,EA001,8.0
1,Aa2,EA001,4.0


In [88]:
var2.head()

,id,title,category
0,EA001,Subsistence economy: gathering,"Subsistence, Economy"
1,EA002,Subsistence economy: hunting,"Subsistence, Economy"
2,EA003,Subsistence economy: fishing,"Subsistence, Economy"
3,EA004,Subsistence economy: animal husbandry,"Subsistence, Economy"
4,EA005,Subsistence economy: agriculture,"Subsistence, Economy"


In [110]:
codes.head(3)

,var_id,code,description,name,cat
0,EA001,0.0,Zero to 5 percent dependence,0-5%,EA0010.0
1,EA001,1.0,6 to 15 percent dependence,6-15%,EA0011.0
2,EA001,2.0,16 to 25 percent dependence,16-25%,EA0012.0


In [111]:
codes[codes.var_id == "EA202"]

,var_id,code,description,name,cat
755,EA202,NaN,Missing data,Missing data,EA202nan


### Merge relational databases

In [167]:
t1 = ea2.merge(soc2, left_on = 'soc_id', right_on = 'id')
t2 = t1.merge(var2, left_on = 'var_id', right_on ='id')
t2['cat'] = t2.var_id.apply(str) + t2.code.apply(str)
codes['cat'] = codes.var_id.apply(str) + codes.code.apply(str)
t3 = t2.merge(codes, left_on = 'cat', right_on = 'cat', how = 'left')
t3['fname'] = np.where(t3.title == 'Population size', t3.code_x, t3.name)
t4 = t3[['pref_name_for_society','title','fname']]

In [171]:
t4.shape

(121354, 3)

In [173]:
t3['fname'] = np.where(t3.title == 'Population size', t3.code_x, t3.name)

In [181]:
t3[t3.title == 'Population size'].shape

(1291, 17)

In [182]:
t4[t4.fname.isnull()].shape

(338, 3)

#### Reshape database

In [ ]:
t4.pivot(index='pref_name_for_society',columns='idx')[['title','fname']]

#### Encode features to a sparce matrix

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict

fullcat = df[['categoria','mes','dia']].copy()
d = defaultdict(LabelEncoder)
fullLe = fullcat.apply(lambda x: d[x.name].fit_transform(x))

cat = OneHotEncoder().fit(fullLe)
cat2 = cat.transform(fullLe)

#### Estimate similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# calcular similitud coseno

simil = {}
matrix = cosine_similarity(tfs, tfs)

for i in range(0, len(matrix)):
    t= matrix[i]
    t2 = numpy.argpartition(t, -2)[-2]
    simil['file' + str(i)] = {'file' + str(t2):{t[t2]}}